# Links

A *colormap* is a palette constructed by sampling some *smooth, linear* function between two end colors. Colormaps are generally used for 2-D or 3-D plots, where the color serves as an extra \"dimension\". This is implemented in matplotlib with the [LinearSegmentedColormap](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.colors.LinearSegmentedColormap.html), and also with the special ProPlot [PerceptuallyUniformColormap](https://proplot.readthedocs.io/en/latest/api/proplot.styletools.PerceptuallyUniformColormap.html#proplot.styletools.PerceptuallyUniformColormap) class. Use this notebook to experiment with different colormaps on sample datasets.

The following online GUI tools can be used to generate perceptually uniform colormaps.

* [cmOcean](https://matplotlib.org/cmocean/)
* [SciVisColor](https://sciviscolor.org/home/colormaps/)
* [Fabio Crameri](http://www.fabiocrameri.ch/colourmaps.php)
* [Chroma.js](https://gka.github.io/palettes/)
* [Chroma.js fork](http://tristen.ca/hcl-picker/#/hlc/12/0.99/C6F67D/0B2026)
* [HCLWizard](http://hclwizard.org:64230/hclwizard/)

You can also programatically generate perceptually uniform colormaps with [ProPlot](https://proplot.readthedocs.io/).

# Colorblind friendly
Something we didn't get into -- you generally want your colormaps to be colorblind friendly. A great tool for testing this is the [Color Oracle](https://colororacle.org) app. In the future, you may also be able to use ProPlot to test the colorblind-friendliness of colormaps.

# Online colormaps 
The below cell uses ProPlot to [register colormaps](https://proplot.readthedocs.io/en/latest/colors.html#adding-online-color-colormaps) online. Try plotting some sample data with them!

In [ ]:
import os
import proplot as plot


# Paste a list of hex strings in the below string
name = 'tmp'
hexdata = """
#000000 #ffffff
"""
with open(os.path.expanduser(f'~/.proplot/cmaps/{name}.hex'), 'w') as f:
    f.write(hexdata)
    
# Paste a 3-column table of RGB data in the below string
name = 'tmp'
rgbdata = """
0, 0, 0
1, 1, 1
"""
with open(os.path.expanduser(f'~/.proplot/cmaps/{name}.rgb'), 'w') as f:
    f.write(rgbdata)
    
# Paste a 4-column table of "colormap coordintes" and RGB data in the below string
# This time the first column is a colormap coordinate
name = 'tmp'
xrgbdata = """
0, 0, 0, 0
1, 1, 1, 1
"""
with open(os.path.expanduser(f'~/.proplot/cmaps/{name}.xrgb'), 'w') as f:
    f.write(xrgbdata)
    
# Paste XML data in the below string
# Some websites only save colormap data as XML strings
name = 'tmp'
xmldata = """
<ColorMap>
  <Point x="0" r="0" g="0" b="0"/>
  <Point x="1" r="1" g="1" b="1"/>
</ColorMap>
"""
with open(os.path.expanduser(f'~/.proplot/cmaps/{name}.xml'), 'w') as f:
    f.write(xmldata)
    
# Load new colormaps into ProPlot
plot.register_cmaps()

In [ ]:
# Breakdown of your colormap
f = plot.show_channels('tmp')

# ProPlot colormaps

The below cells use ProPlot to design custom colormaps using some helper functions.

In [ ]:
# Plot random data
import proplot as plot
import numpy as np

f, ax = plot.subplots()
ax.pcolor(np.random.rand(10,10), cmap='tmp', colorbar='b')

# Try out your colormaps

We've added a few observational datasets to the repository to try out your colormaps. Just replace the `cmap=...` with the name of your colormap from above.

## Sequential Colormaps

These examples anticipate a sequential colormap. They are mainly comprised of climatologies.

### Atmospheric Data

### Oceanography Data (Sea Surface Temperatures)

In [ ]:
import xarray as xr

SST = xr.open_dataarray('../data/SST.HadNOAA.1970-2010.1deg.nc')

f, ax = plot.subplots(proj='eqearth', axwidth='12cm')

p = ax.contourf(SST.lon, SST.lat, SST, levels=plot.arange(-2, 30, 2),
                cmap='Fire')

f.colorbar(p, loc='b', ticks=plot.arange(-2, 30, 2),
           label='temperature [$^{o}$C]')

ax.format(land=True, title='Sea Surface Temperature Climatology')

# Uncomment this to save your figure. Feel free to rename it.
# f.savefig('../figures/SST.climatology.png')

### Terrestrial Data

### Sea Ice Data

In [ ]:
import xarray as xr

# Plot sea ice data
ice_data = []
for hemisphere, season in zip(['NH', 'NH', 'SH', 'SH'], 
                              ['fm', 'on', 'fm', 'on']):
    filepath = f'../data/ICESat_gridded_mean_thickness_{hemisphere}_{season}.interp0.5x0.5.nc'
    sea_ice = xr.open_dataset(filepath)
    ice_data.append(sea_ice)
ice_data = xr.concat(ice_data, 'location')
ice_data['location'] = ['NH_fm', 'NH_on', 'SH_fm', 'SH_on']

# Coarsen to a 1x1 degree grid.
lat = ice_data['t_lat'].coarsen({'t_lat': 2, 't_lon': 2}).mean()
lon = ice_data['t_lon'].coarsen({'t_lat': 2, 't_lon': 2}).mean()
ice_thk = ice_data['HI'].reset_coords(drop=True).coarsen({'t_lat': 2, 't_lon': 2}).mean()

In [ ]:
f, axs = plot.subplots(ncols=2, proj='npstere', axwidth='6cm')

for i, label in zip([0, 1], ['NH Feb-Mar', 'NH Oct-Nov']):
    p = axs[i].contourf(lon, lat, ice_thk.isel(location=i), cmap='Glacial',
                    levels=plot.arange(0, 5, 0.25))
    axs[i].format(land=True, boundinglat=60, latlines=10, title=label)

f.colorbar(p, loc='b', label='Sea Ice Thickness [m]',
           ticks=plot.arange(0, 5, 0.5))

# Uncomment this to save your figure. Feel free to rename it.
# f.savefig('../figures/NP.sea.ice.png')

In [ ]:
f, axs = plot.subplots(ncols=2, proj='spstere', axwidth='6cm')

for i, label in zip([0, 1], ['SH Feb-Mar', 'NH Oct-Nov']):
    p = axs[i].contourf(lon, lat, ice_thk.isel(location=i+2), cmap='Glacial',
                    levels=plot.arange(0, 2, 0.1))
    axs[i].format(land=True, boundinglat=-50, latlines=10, title=label)

f.colorbar(p, loc='b', label='Sea Ice Thickness [m]',
           ticks=plot.arange(0, 2, 0.2))

# Uncomment this to save your figure. Feel free to rename it.
# f.savefig('../figures/SH.sea.ice.png')

## Diverging Colormaps

These examples anticipate a diverging colormap. They are mainly comprised of anomalies. 